# Rule Learner algorithm (PRISM)

Use lecture slides and/or book chapters to understand the PRISM algorithm. 

Then implement the algorithm in this notebook. Implement it step-by-step, add explanations of every step, keep the code clean and modular. Finally, test the correctness of your algorithm on the contact lenses [dataset](contact_lenses.csv) discussed in class, to see if it produces the same rules.

If the PRISM algorithm is designed to cover the entire dataset, then the rules can be organized into a decision table, and used for classification. However, in this project, we are more interested in extracting small nuggets of knowledge: the most reliable rules - with significant coverage and high accuracy. 

## 1. Notes and suggestions

- Advice: work on the  algorithm implementation in IDE (such as Pycharm). This will allow you easily find and fix bugs. 
- After your algorithm works, you can copy it to this notebook in pieces and explain each part.
- Use `pandas` and `numpy` whenever possible to make the program faster.
- Use list comprehensions instead of loops - these run 10 times faster than native Python loops.
- __Important:__ you do not have to use the starter logic below. You can delete all of this and write your own code from scratch. The goal was to help you, not to constrain your creativity to my way of thinking.


## 2. Algorithm implementation

First, we need a datastructure to store each rule. 
Possible implementation of required data types is given below. You are free to either reuse them or create your own.

Each Rule consists of antecedent (Left Hand Side) and consequent (Right Hand Side). The LHS includes multiple conditions joined with AND, and RHS is a class label. The Rule also needs to store its accuracy and coverage.

In [ ]:
class Rule:
    def __init__(self, class_label):
        self.conditions = []  # list of conditions
        self.class_label = class_label  # rule class

    def add_condition(self, condition):
        self.conditions.append(condition)

    def set_params(self, accuracy, coverage):
        self.accuracy = accuracy
        self.coverage = coverage
    
    # Human-readable printing of this Rule
    def __repr__(self):
        return "If {} then {}. Coverage:{}, accuracy: {}".format(self.conditions, self.class_label,
                                                                 self.coverage, self.accuracy)

The list of conditions contains several objects of class Condition. 

Each condition includes the _attribute name_ and the _value_. 

If the _value_ is numeric, then the condition also includes an additional field `true_false` which means the following: *if true_false == True then values are >= value* and  *if true_false == False then values are < value*.

In [ ]:
class Condition:
    def __init__(self, attribute, value, true_false = None):
        self.attribute = attribute
        self.value = value
        self.true_false = true_false

    def __repr__(self):
        if self.true_false is None:
            return "{}={}".format(self.attribute, self.value)
        else:
            return "{}>={}:{}".format(self.attribute, self.value, self.true_false)

Next comes the `learn_one_rule` algorithm. The required parameters are the names of the columns, the current subset of data, and the class label (RHS of this rule). The optional parameters are thresholds `min_coverage` and `min_accuracy`. 

In addition, sometimes we can pass an already existing Rule in order to learn a more refined condition. Now, if the Rule can be improved, we will return the new rule and the subset of data covered by the Rule. If we could not improve it - we return the original Rule and the original covered subset (`prev_covered`).

In [ ]:
def learn_one_rule(columns, data, class_label, 
                   prev_rule=None, min_coverage=30,
                   min_accuracy=0.6, prev_covered=None):
    current_subset = data.copy()

    current_rule = prev_rule
    current_accuracy = 0
    current_coverage = None
    covered_subset = None
    
    if current_rule is not None:
        current_accuracy = current_rule.accuracy
        current_coverage = current_rule.coverage
        covered_subset = prev_covered.copy()
    
    best_col = None
    best_val = None
    true_false = None

    # find the rule with the highest accuracy by checking different possible conditions
    for col in columns[:-1]:
        # Extract unique values from the column
        unique_vals = current_subset[col].unique().tolist()
        
        # Consider each unique value in turn
        for val in unique_vals:
            
            # The treatment is different for numeric and categorical attributes
            if isinstance(val, int) or isinstance(val, float):
                # Here we construct 2 conditions: 
                # if actual value >= val and if actual value < val
                
                # Compute:
                # total of >= val
                # correct_total of >= val that have class_label
                # total of < val
                # correct_total of < val that have class_label
               
                
                # If any of the correct_total >= min_coverage
                # we compare their accuracy and choose the one with with higher accuracy
                # in case of accuracy ties we choose the one with greater coverage
                                
                # Now we see if the best accuracy is above threshold 
                # and is better than the previous accuracy (rule improved)
                # and remember this condition
              
            else:
                # For categorical attributes - this is just a single condition: if actual value == val
                total = len(current_subset[current_subset[col] == val])
                # total of == val
                # correct_total of == val that have class_label

                # compute accuracy and coverage as before

            # print(best_col, best_val, current_accuracy, current_coverage)

    # If we managed to improve the rule accuracy
    if best_col is not None:
        # If the rule does not exist - create it
        if current_rule is None:
            current_rule = Rule(class_label)
        # Create a condition based on best_col, best_val and true_false
        condition = Condition(best_col, best_val, true_false)
        current_rule.add_condition(condition)
        current_rule.set_params(current_accuracy, current_coverage)
        
        # Generate a subset covered by this rule
        if true_false is None:
            covered_subset = current_subset[current_subset[best_col] == best_val]
        else:
            if true_false == True:
                covered_subset = current_subset[current_subset[best_col] >= best_val]
            else:
                covered_subset = current_subset[current_subset[best_col] < best_val]
    
    # If we did not refine the rule - return original rule and the original covered set
    return (current_rule, covered_subset, best_col)

Finally, the main algorithm `learn_rules` takes as parameters list of columns, with the last column representing the class label, and the original data in form of pandas dataframe. Optionally, you can pass the list of classes in order that you are interested to explore first. Two optional threshold parameters `min_coverage` and `min_accuracy` set up the conditions of rule's validity for a given dataset.

In [ ]:
import pandas as pd
import numpy as np

def learn_rules (columns, data, classes=None, 
                 min_coverage = 30, min_accuracy = 0.6):
    # List of final rules
    rules = []
    
    # If list of classes of interest is not provided - it is extracted from the last column of data
    if classes is not None:
        class_labels = classes
    else:
        class_labels = data[columns[-1]].unique().tolist()

    current_data = data.copy()
    
    # This follows the logic of the original PRISM algorithm
    # It processes each class in turn. Because for high accuracy 
    # the rules generated are disjoint with respect to class label
    # this is not a problem when we are just interested in rules themselves - not classification
    # For classification the order in which the rules are discovered matters, and we should 
    # process all the classes at the same time
    for class_label in class_labels:
        done = False
        while len(current_data) > min_coverage and not done:
            # Learn a rule with a single condition
            
            # If the best rule does not pass the coverage threshold - we are done with this class
            if rule is None:
                break

            # If we get the rule with coverage above threshold
            # We try to refine this rule
            if rule is not None:
                # try to improve the rule using the same learn_one_rule and passing existing rule as parameter
                # here need another loop which stops when accuracy is not improving    
                 
                # done with this rule
                if rule.accuracy >= min_accuracy:
                    rules.append(rule)

                    # remove rows covered by this rule
                    # you have to remove the rows where all of the conditions hold
                   
                else:
                    done = True
            else:
                done = True
                
    return rules

## 3. Correctness test  

Test your algorithm on the original dataset from the PRISM paper.

The dataset was downloaded from [here](https://archive.ics.uci.edu/ml/datasets/Lenses). The CSV version is included in this repository.

**Attribute Information**:
3 Classes:
- __1__ : the patient should be fitted with __hard__ contact lenses,
- __2__ : the patient should be fitted with __soft__ contact lenses,
- __3__ : the patient should __not__ be fitted with contact lenses.

Features:
1. age of the patient: (1) young, (2) pre-presbyopic, (3) presbyopic
2. spectacle prescription:  (1) myope, (2) hypermetrope
3. astigmatic:     (1) no, (2) yes
4. tear production rate:  (1) reduced, (2) normal

Presbyopia is physiological insufficiency of accommodation associated with the aging of the eye that results in progressively worsening ability to focus clearly on close objects. So presbiopic means old.

Hypermetropia: far-sightedness, also known as long-sightedness - cannot see close.
Myopia: nearsightedness - cannot see at distance.

In [ ]:
import pandas as pd
data_file = "contact_lenses.csv"
data = pd.read_csv(data_file, index_col=['id'])
data.columns

We can replace numbers with actual values - for clarity.

In [ ]:
# classes
conditions = [ data['lenses type'].eq(1), data['lenses type'].eq(2), data['lenses type'].eq(3)]
choices = ["hard","soft","none"]

data['lenses type'] = np.select(conditions, choices)

# age groups
conditions = [ data['age'].eq(1), data['age'].eq(2), data['age'].eq(3)]
choices = ["young","medium","old"]

data['age'] = np.select(conditions, choices)

# spectacles
conditions = [ data['spectacles'].eq(1), data['spectacles'].eq(2)]
choices = ["nearsighted","farsighted"]

data['spectacles'] = np.select(conditions, choices)

# astigmatism
conditions = [ data['astigmatism'].eq(1), data['astigmatism'].eq(2)]
choices = ["no","yes"]

data['astigmatism'] = np.select(conditions, choices)

# tear production rate
conditions = [ data['tear production rate'].eq(1), data['tear production rate'].eq(2)]
choices = ["reduced","normal"]

data['tear production rate'] = np.select(conditions, choices)

In [ ]:
column_list = data.columns.to_numpy().tolist()
rules = learn_rules (column_list, data, None, 1, 0.95)
for rule in rules[:20]:
    print(rule)

My results are given below for comparison:

If [tear production rate=reduced] then none. Coverage:12, accuracy: 1.0

If [astigmatism=no, spectacles=farsighted] then soft. Coverage:3, accuracy: 1.0

If [astigmatism=no, age=young] then soft. Coverage:1, accuracy: 1.0

If [astigmatism=no, age=medium] then soft. Coverage:1, accuracy: 1.0

If [age=young] then hard. Coverage:2, accuracy: 1.0

If [spectacles=nearsighted, astigmatism=yes] then hard. Coverage:2, accuracy: 1.0

Copyright &copy; 2020 Marina Barsky. All rights reserved.